# USDe Morpho Benchmarks and Reallocation

In this Jupyter notebook, we will automate the process of analyzing and reallocating USDe liquidity within Morpho's financial pools. 

Our primary focus will lie in assessing the current allocation of USDe, the utilization rates, borrowing rates, optimal rates, and capped rates across different liquidity pools distinguished by their Loan-to-Value (LTV) ratios. We aim to ensure that pool allocations are optimized both for the stable functioning of the overall market and for better returns for liquidity providers and borrowers.

1. **Data Analysis**: We will start by performing a data analysis on the current status of various USDe pools. This will include processing important metrics such as total supply, borrow rates, and optimal rates to identify pools that may require attention or reallocation.

2. **Manual Reallocation Criteria**: Based on the analysis, we'll assess the necessity for manual reallocations in accordance with established criteria for active and inactive pools, including factors like minimum balances, maximum utilization, and the safety margin for withdrawing portions of the allocated USDe supply.

In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import json
from data_manipulation import *
from api import *

pd.set_option('display.max_columns', None)

## Manual Reallocation Analysis

Defined Pools we look at for allocation and necessary values

In [2]:
pool_keys = [
    'USDe 94.50%',
    'USDe 91.50%',
    'USDe 86%',
    'USDe 77%',
    'sUSDe 94.50%',
    'sUSDe 91.50%',
    'sUSDe 86%',
    'sUSDe 77%',
    'PT-sUSDe-24Oct2024 86%',
    'PT-sUSDe-26Dec2024 91.5%',
    'PT-sUSDe-27Mar2025 91.5%'
]
idle_key = [
    'Idle',
]
idle_col = [
    'LLTV',
    'Total Supply', 
    'Maker Allocation',     
    'Utilization', 
    'Borrow Rate',              
]
columns_ordered = [
    'Status', 
    'LLTV',
    'Total Supply', 
    'Maker Allocation', 
    'Utilization', 
    'Borrow Rate',
    'Supply Cap',
    'Total Borrow', 
    'Maker Borrow', 
    'Optimal Rate', 
    'Capped Borrow Rate', 
    'SSR',
    'Fixed Spread',
    'Fixed Spread Rate Value',
    'Fixed Slope per 100M',
    'Fixed Slope',
    'Proportional Spread',
    'Proportional Spread Rate Value',
    'Proportional Spread Fixed Value',
    'Proportional Slope per 100M',
    'Proportional Slope',
    'Proportional Slope Fixed Value',
    'Low Target Threshold',
    'High Target Threshold',
    'Target Borrow Rate',
    'Min Borrow Rate',
    'Max Borrow Rate',
    'Utilization Where Rate Equal to SSR',
    'SSR Adjustment',
    'Total Supply After SSR Adjustment',
    'Maker Supply After SSR Adjustment',
    'Utilization After SSR Adjustment',
    'Inactive Withdrawal',
    'Utilization Where Rate Equal To Min Target',
    'Active Withdrawal',
    'Utilization Where Rate Equal To Max Target',
    'Active Deposits',
    'Manual Adjustment',
    'Total Change',
    'Final Allocation',
    'Final Supply',
    'Final Utilization',
    'Final Borrow Rate',
    'Final Capped Rate',
    'Maker Borrow at Old Utilization',
    'Borrow Rate Change',                   
]

pools_df = pd.DataFrame(index=pool_keys, columns=columns_ordered)
idle_df = pd.DataFrame(index=idle_key, columns=idle_col)
pools_df

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,SSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to SSR,SSR Adjustment,Total Supply After SSR Adjustment,Maker Supply After SSR Adjustment,Utilization After SSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 91.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 86%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 77%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 94.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 91.50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 86%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 77%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-24Oct2024 86%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-26Dec2024 91.5%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
idle_df

,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate
Idle,NaN,NaN,NaN,NaN,NaN


### Retrieve latest values

Connect to BA API and retrieve the data for the pools we are interested in. First define pools by their uuid, to retrieve the data.

In [4]:
market_to_pool = {
    "db760246f6859780f6c1b272d47a8f64710777121118e56e0cdb4b8b744a3094": 'USDe 94.50%',
    "8e6aeb10c401de3279ac79b4b2ea15fc94b7d9cfc098d6c2a1ff7b2b26d9d02c": 'USDe 91.50%',
    "c581c5f70bd1afa283eed57d1418c6432cbff1d862f94eaf58fdd4e46afbb67f": 'USDe 86%',
    "fd8493f09eb6203615221378d89f53fcd92ff4f7d62cca87eece9a2fff59e86f": 'USDe 77%',
    "e475337d11be1db07f7c5a156e511f05d1844308e66e17d2ba5da0839d3b34d9": 'sUSDe 94.50%',
    "1247f1c237eceae0602eab1470a5061a6dd8f734ba88c7cdc5d6109fb0026b28": 'sUSDe 91.50%',
    "39d11026eae1c6ec02aa4c0910778664089cdd97c3fd23f68f7cd05e2e95af48": 'sUSDe 86%',
    "42dcfb38bb98767afb6e38ccf90d59d0d3f0aa216beb3a234f12850323d17536": 'sUSDe 77%',
    "8f46cd82c4c44a090c3d72bd7a84baf4e69ee50331d5deae514f86fe062b0748": 'PT-sUSDe-24Oct2024 86%',
    "e37784e5ff9c2795395c5a41a0cb7ae1da4a93d67bfdd8654b9ff86b3065941c": 'PT-sUSDe-26Dec2024 91.5%',
    "5e3e6b1e01c5708055548d82d01db741e37d03b948a7ef9f3d4b962648bcbfa7": 'PT-sUSDe-27Mar2025 91.5%'
}

In [5]:
# market_to_pool.keys()

Retrieve the data from API

In [6]:
# Base URL for market data
base_market_url = "https://morpho-api.blockanalitica.com/markets/{}/historic/?days_ago=0&network=ethereum"
# Base URL for wallet data
base_wallet_url = "https://morpho-api.blockanalitica.com/markets/{}/wallets/?network=ethereum&order=-supply&p_size=4&type=suppliers"
# URL for vault data
vaults_url = "https://morpho-api.blockanalitica.com/vaults/0x73e65dbd630f90604062f6e02fab9138e713edd9/pools/?days_ago=0&network=ethereum&order=-supply_usd&p=1&p_size=15"
# URL for SSR rate
ssr_url = "https://info-sky.blockanalitica.com/save/"
# The wallet address we are interested in
target_wallet = "0x73e65dbd630f90604062f6e02fab9138e713edd9"

# Assuming market_to_pool is defined
api_data = ApiBA(base_market_url, base_wallet_url, market_to_pool.keys(), target_wallet, vaults_url, ssr_url)

With data from API populate the dataframe

In [7]:
market_data = PoolDataHandler(pools_df, market_to_pool)
pools_df = market_data.populate_dataframe(api_data.fetch_data())
pools_df


Updated DataFrame:


,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,SSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to SSR,SSR Adjustment,Total Supply After SSR Adjustment,Maker Supply After SSR Adjustment,Utilization After SSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,NaN,0.945,63007,51594,0.2167,0.0029,10000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 91.50%,NaN,0.915,28763294,28635257,0.8995,0.0878,200000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 86%,NaN,0.86,35163902,35163800,0.9202,0.1109,500000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDe 77%,NaN,0.77,383232,383232,0.9288,0.1427,1000000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 94.50%,NaN,0.945,416844,0,0.8973,0.0902,10000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 91.50%,NaN,0.915,27159638,27159638,0.9119,0.1,200000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 86%,NaN,0.86,69917553,69400710,0.8989,0.0652,500000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sUSDe 77%,NaN,0.77,100544,74877,0.8945,0.0472,1000000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-24Oct2024 86%,NaN,0.86,67429,67428,0.9299,0.0632,100000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PT-sUSDe-26Dec2024 91.5%,NaN,0.915,100340372,100340370,0.9354,0.113,100000000,NaN,NaN,NaN,NaN,0.065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#index_to_drop = 'sUSDe 77%PT-sUSDe-24Oct2024 86%'
#pools_df = pools_df.drop(index_to_drop)

To see the allocation in the Idle matket:

In [9]:
idle_market_to_pool = {"57f4e42c0707d3ae0ae39c9343dcba78ff79fa663da040eca45717a9b0b0557f": 'Idle'}
idle_api_data = ApiBA(base_market_url, base_wallet_url, idle_market_to_pool.keys(), target_wallet, vaults_url, ssr_url)
idle_data = PoolDataHandler(idle_df, idle_market_to_pool)
idle_df = idle_data.populate_dataframe(idle_api_data.fetch_data())
idle_df


Updated DataFrame:


,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,SSR
Idle,0.0,38425576,38425576,0.0,0.0,24519928653854223487948685977518604288,0.065


### Reallocation Metaparameters

They are set as done by Monet. Change only in case you know what you are doing!

In [10]:
realloc_metaparm = {
    'active_pool' : {
        'min_balance' : 10000000,
        'max_utilization' : 0.9,
        'max_portion_to_withdraw' : 0.1,
        'allocation_significance_threshold' : 10000,
    },
    'inactive_pool' : {
        'min_balance' : 100000,
        'max_utilization' : 0.93,
        'max_portion_to_withdraw' : 1,
        'allocation_significance_threshold' : 10000,
    },
}

In [11]:
pools_analysis = PoolAnalysis(pools_df, idle_df, realloc_metaparm)

#### Classify market as Active or Inactive

Change the status of the pool you want to change by changing the pool status to `Active` or `Inactive`

In [12]:
pools_df.loc['USDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['USDe 91.50%', 'Status'] = 'Active'
pools_df.loc['USDe 86%', 'Status'] = 'Active'
pools_df.loc['USDe 77%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 94.50%', 'Status'] = 'Inactive'
pools_df.loc['sUSDe 91.50%', 'Status'] = 'Active'
pools_df.loc['sUSDe 86%', 'Status'] = 'Active'
pools_df.loc['sUSDe 77%', 'Status'] = 'Inactive'
pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Status'] = 'Inactive'
pools_df.loc['PT-sUSDe-26Dec2024 91.5%', 'Status'] = 'Active'
pools_df.loc['PT-sUSDe-27Mar2025 91.5%', 'Status'] = 'Active'

## Vault benchmarks

Here you can modify the vault benchmark calculations.

In [13]:
# Tweakable parameters
pools_df['Fixed Spread'] = 1.75 / 100
pools_df['Fixed Slope per 100M'] = 0.65 / 100

pools_df['Proportional Spread'] = 50 / 100
pools_df['Proportional Slope per 100M'] = 3 / 100

pools_df['Low Target Threshold'] = 80 / 100
pools_df['High Target Threshold'] = 105 / 100

# Indirect parameters
pools_df['Fixed Spread Rate Value'] = pools_df['SSR'] + pools_df['Fixed Spread']
pools_df['Fixed Slope'] = pools_df['Fixed Slope per 100M'] / 100000000
pools_df['Proportional Spread Rate Value'] = pools_df['SSR'] * (1 + pools_df['Proportional Spread'])
pools_df['Proportional Spread Fixed Value'] = pools_df['Proportional Spread Rate Value'] - pools_df['SSR']
pools_df['Proportional Slope'] = pools_df['Proportional Slope per 100M'] / 100000000
pools_df['Proportional Slope Fixed Value'] = pools_df['Proportional Spread Rate Value'] * pools_df['Proportional Slope per 100M']

In [14]:
#pools_analysis.define_active_or_inactive()

### Manual Adjustment

If you want to add a manual adjustment, do it here by uncommenting the line of code with pool you want to adjust and change the number inside the parenthesis.

In [15]:
#pools_df.loc['USDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['USDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 94.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 91.50%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['sUSDe 77%', 'Manual Adjustment'] = int(0)
#pools_df.loc['PT-sUSDe-24Oct2024 86%', 'Manual Adjustment'] = int(0)
#pools_df.loc['PT-sUSDe-26Dec2024 91.5%', 'Manual Adjustment'] = int(0)
#pools_df.loc['PT-sUSDe-27Mar2025 91.5%', 'Manual Adjustment'] = int(0)

### Reallocation Dataframe

Get the final table with all data. For easier overview focus on the following tables

In [16]:
pools_analysis = PoolAnalysis(pools_df, idle_df, realloc_metaparm)

In [17]:
pools_analysis.update_pool_dataframe()

,Status,LLTV,Total Supply,Maker Allocation,Utilization,Borrow Rate,Supply Cap,Total Borrow,Maker Borrow,Optimal Rate,Capped Borrow Rate,SSR,Fixed Spread,Fixed Spread Rate Value,Fixed Slope per 100M,Fixed Slope,Proportional Spread,Proportional Spread Rate Value,Proportional Spread Fixed Value,Proportional Slope per 100M,Proportional Slope,Proportional Slope Fixed Value,Low Target Threshold,High Target Threshold,Target Borrow Rate,Min Borrow Rate,Max Borrow Rate,Utilization Where Rate Equal to SSR,SSR Adjustment,Total Supply After SSR Adjustment,Maker Supply After SSR Adjustment,Utilization After SSR Adjustment,Inactive Withdrawal,Utilization Where Rate Equal To Min Target,Active Withdrawal,Utilization Where Rate Equal To Max Target,Active Deposits,Manual Adjustment,Total Change,Final Allocation,Final Supply,Final Utilization,Final Borrow Rate,Final Capped Rate,Maker Borrow at Old Utilization,Borrow Rate Change
USDe 94.50%,Inactive,0.945,63007,51594,0.2167,0.0029,10000000,13653,11179,0.0067,0.0029,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,1,0,63007,51594,0.2167,0,1,0,1,0,0,0,51594,63007,0.2167,0.0029,0.0029,11180,0.0
USDe 91.50%,Active,0.915,28763294,28635257,0.8995,0.0878,200000000,25872582,25757412,0.0878,0.0878,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.5884,0,28763294,28635257,0.8995,0,0.8945,0,0.9102,0,0,0,28635257,28763294,0.8995,0.0878,0.0878,25757413,0.0
USDe 86%,Active,0.86,35163902,35163800,0.9202,0.1109,500000000,32357822,32357728,0.0691,0.0691,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.8288,0,35163902,35163800,0.9202,0,0.9088,0,0.922,0,0,0,35163800,35163902,0.9202,0.111,0.0691,32357728,0.0001
USDe 77%,Inactive,0.77,383232,383232,0.9288,0.1427,1000000000,355945,355945,0.0766,0.0766,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.7183,0,383232,383232,0.9288,-495,0.9047,0,0.9166,0,0,0,383232,383232,0.9288,0.1428,0.0766,355945,0.0001
sUSDe 94.50%,Inactive,0.945,416844,0,0.8973,0.0902,10000000,374034,0,0.0904,0.0902,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.5628,0,416844,0,0.8973,0,0.8602,0,0.909,0,0,0,0,416844,0.8973,0.0902,0.0902,0,0.0
sUSDe 91.50%,Active,0.915,27159638,27159638,0.9119,0.1,200000000,24766873,24766873,0.0737,0.0737,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.7583,0,27159638,27159638,0.9119,0,0.9062,0,0.9185,0,0,0,27159638,27159638,0.9119,0.1,0.0737,24766873,0.0
sUSDe 86%,Active,0.86,69917553,69400710,0.8989,0.0652,500000000,62848888,62384297,0.0653,0.0652,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.8945,0,69917553,69400710,0.8989,0,0.9113,-85455,0.9252,0,0,-85455,69315255,69832098,0.9,0.0653,0.0653,62307482,0.0001
sUSDe 77%,Inactive,0.77,100544,74877,0.8945,0.0472,1000000000,89936,66977,0.0474,0.0472,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.9124,-544,100000,74333,0.8994,0,0.9281,0,0.9473,0,0,0,74877,100544,0.8945,0.0472,0.0472,66977,0.0
PT-sUSDe-24Oct2024 86%,Inactive,0.86,67429,67428,0.9299,0.0632,100000000,62702,62701,0.0333,0.0333,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.9317,0,67429,67428,0.9299,0,0.9542,0,0.9815,0,0,0,67428,67429,0.9299,0.0632,0.0333,62701,0.0
PT-sUSDe-26Dec2024 91.5%,Active,0.915,100340372,100340370,0.9354,0.113,100000000,93858383,93858381,0.0548,0.0548,0.065,0.0175,0.0825,0.0065,6.500000e-11,0.5,0.0975,0.0325,0.03,3.000000e-10,0.002925,0.8,1.05,0.1092,0.0874,0.1147,0.9062,0,100340372,100340370,0.9354,0,0.9198,0,0.9364,0,0,0,100340

Save the data if you want to perform later analysis or visualizations on this

In [18]:
#pools_analysis.pool_df.to_json(f'reallocations_{datetime.datetime.now().strftime('%Y_%m_%d')}.json')

### Average Utilization, Net and Running Change

Some basic statistics for reallocation

In [19]:
average_utilization = pools_analysis.pool_df['Utilization'].mean()
print(f'The average utilization of the pools is: {average_utilization * 100:.2f}%')

The average utilization of the pools is: 85.03%


The net changes based on the type of adjusment are as follows

In [20]:
net_change_df = pools_analysis.pool_df[['SSR Adjustment', 'Inactive Withdrawal', 'Active Withdrawal', 'Active Deposits', 'Manual Adjustment']].agg('sum').to_frame().T
net_change_df.columns = ['SSR Adjustment Sum', 'Inactive Withdrawal Sum', 'Active Withdrawal Sum', 'Active Deposits Sum', 'Manual Adjustment Sum']
net_change_df = net_change_df.round(0).astype(int)

net_change_df

,SSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-544,-495,-85455,0,0


The running total of changes by the type of adjustment are as follows, which will result in new adjusted Idle supply

In [21]:
cumulative_sums = net_change_df.cumsum(axis=1).iloc[-1]
cumulative_sums_df = pd.DataFrame({
    'SSR Adjustment Sum': [cumulative_sums['SSR Adjustment Sum']],
    'Inactive Withdrawal Sum': [cumulative_sums['Inactive Withdrawal Sum']],
    'Active Withdrawal Sum': [cumulative_sums['Active Withdrawal Sum']],
    'Active Deposits Sum': [cumulative_sums['Active Deposits Sum']],
    'Manual Adjustment Sum': [cumulative_sums['Manual Adjustment Sum']]
})
cumulative_sums_df = cumulative_sums_df.round(0).astype(int)

cumulative_sums_df

,SSR Adjustment Sum,Inactive Withdrawal Sum,Active Withdrawal Sum,Active Deposits Sum,Manual Adjustment Sum
0,-544,-1039,-86494,-86494,-86494


## Manual Reallocation Dataframe

The following table shows the change and new allocated supply with wei allocation

In [22]:
new_pool_data = {
    'Net Change': pools_df['Total Change'],
    'New Allocation Supply': pools_df['Final Allocation']
}
manual_realloaction_pools = pd.DataFrame(new_pool_data)
manual_realloaction_pools['New Allocation Wei'] = manual_realloaction_pools['New Allocation Supply'] * pow(10, 18)

In [23]:
# Create a styled DataFrame
styled_df = manual_realloaction_pools.style.format({
    'Net Change': '{:.0f}',
    'New Allocation Supply': '{:.0f}',
    'New Allocation Wei': '{:.0f}'
}).map(PoolAnalysis.color_net_change, subset=['Net Change'])

styled_df

,Net Change,New Allocation Supply,New Allocation Wei
USDe 94.50%,0,51594,51594000000000001572864
USDe 91.50%,0,28635257,28635257000000000565968896
USDe 86%,0,35163800,35163800000000000614465536
USDe 77%,0,383232,383232000000000000000000
sUSDe 94.50%,0,0,0
sUSDe 91.50%,0,27159638,27159637999999998194876416
sUSDe 86%,-85455,69315255,69315255000000004018405376
sUSDe 77%,0,74877,74877000000000002883584
PT-sUSDe-24Oct2024 86%,0,67428,67427999999999998951424
PT-sUSDe-26Dec2024 91.5%,0,100340370,100340370000000006458703872


In [24]:
print(f'The total net change in allocation is: ${manual_realloaction_pools["Net Change"].sum():.0f}')

The total net change in allocation is: $-85455


## Pool Overview Dataframe

A brief statistics for pool overview

In [25]:
pool_stats = [
    'Total Non-Idle Allocation',
    'Supply Weighted LLTV',
    'Supply Weighted sUSDe',
    'Average Borrow Rate',
    'Average Capped Rate',
    'Rate at Prior Equilibrium',
]
columns_stats = [
    'Current',
    'Future',
    'Change'                 
]

pool_overview = pd.DataFrame(index=pool_stats, columns=columns_stats)
statistics = PoolOverview(pools_df, pool_overview)

In [26]:
statistics.update_pool_overview()

,Current,Future,Change
Total Non-Idle Allocation,361576349,361490894,-0.000236
Supply Weighted LLTV,0.898905,0.898914,0.000010
Supply Weighted sUSDe,0.267261,0.267088,-0.000648
Average Borrow Rate,0.099529,0.099557,0.000288
Average Capped Rate,0.065584,0.065603,0.000287
Rate at Prior Equilibrium,0.099529,0.099557,0.000288


In [27]:
pool_styled_df = pool_overview.style.format({
    'Current': '{:.3f}',
    'Future': '{:.3f}',
    'Change': '{:.2f}%'
}).map(PoolAnalysis.color_net_change, subset=['Change'])

pool_styled_df

,Current,Future,Change
Total Non-Idle Allocation,361576349.000,361490894.000,-0.00%
Supply Weighted LLTV,0.899,0.899,0.00%
Supply Weighted sUSDe,0.267,0.267,-0.00%
Average Borrow Rate,0.100,0.100,0.00%
Average Capped Rate,0.066,0.066,0.00%
Rate at Prior Equilibrium,0.100,0.100,0.00%


### Current Debt Ceiling

In [28]:
current_ddm_target_debt = int(pool_overview.at['Total Non-Idle Allocation', 'Current'] + idle_df.at['Idle', 'Total Supply'])

print(f'The current DDM target debt is: ${current_ddm_target_debt}')

future_ddm_target_debt = math.ceil((int(pool_overview.at['Total Non-Idle Allocation', 'Future'])+ 5000000)/ 25000000) * 25000000

print(f'The future DDM target debt is: ${future_ddm_target_debt}')

The current DDM target debt is: $400001925
The future DDM target debt is: $375000000


## JSON Logic

Include only pools that change in allocation

### Allocation to USDe

[
MakerDAO: Dai Token (DAI),
Ethena: USDe (USDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E16)
],
Allocation WEI

### Allocation to sUSDe

[
MakerDAO: Dai Token (DAI),
Ethena: Staked USDe (sUSDe),
Morpho: MorphoChainlinkOracleV2,
Morpho: AdaptiveCurveIRM,
LLTV (times 10E16)
]
Allocation WEI

### Everything else goes to the idle market

[
MakerDAO: Dai Token (DAI),
Burn Address,
Burn Address,
Burn Address,
0
]
Uint256 Maximum Value

In [29]:
manual_reallocation_pools = manual_realloaction_pools

manual_reallocation_pools.index = [
    'USDe 94.50%', 'USDe 91.50%', 'USDe 86%', 'USDe 77%', 
    'sUSDe 94.50%', 'sUSDe 91.50%', 'sUSDe 86%', 'sUSDe 77%', 
    'PT-sUSDe-24Oct2024 86%', 'PT-sUSDe-26Dec2024 91.5%', 'PT-sUSDe-27Mar2025 91.5%'
]

# Define the base address values - Source: https://docs.morpho.org/addresses/
base_addresses = {
    "USDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x4c9EDD5852cd905f086C759E8383e09bff1E68B3", # USDe
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "sUSDe": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x9D39A5DE30e57443BfF2A8307A4256c8797A3497", # sUSDe
        "0x5D916980D5Ae1737a8330Bf24dF812b2911Aae25", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "PT-sUSDe-24Oct2024": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0xAE5099C39f023C91d3dd55244CAFB36225B0850E", # PT-sUSDe-24Oct2024
        "0xaE4750d0813B5E37A51f7629beedd72AF1f9cA35", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "PT-sUSDe-26Dec2024 91.5%": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0xEe9085fC268F6727d5D4293dBABccF901ffDCC29", # PT-sUSDe-26Dec2024
        "0x81E5E28F33D314e9211885d6f0F4080E755e4595", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "PT-sUSDe-27Mar2025 91.5%": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0xe00bd3df25fb187d6abbb620b3dfd19839947b81", # PT-sUSDe-27Mar2025
        "0x38d130cEe60CDa080A3b3aC94C79c34B6Fc919A7", # MorphoChainlinkOracleV2
        "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC" # AdaptiveCurveIRM
    ],
    "Idle": [
        "0x6B175474E89094C44Da98b954EedeAC495271d0F", # DAI
        "0x0000000000000000000000000000000000000000", # Idle
        "0x0000000000000000000000000000000000000000", # MorphoChainlinkOracleV2
        "0x0000000000000000000000000000000000000000" # AdaptiveCurveIRM
    ]
}

# Function to determine the pool type based on the allocation type (in the index)
def determine_pool_type(allocation_type):
    # Extract the part before the percentage
    asset_type = allocation_type.split()[0]  # Gets "USDe", "sUSDe", or "PT-sUSDe-24Oct2024"
    
    if asset_type == "USDe":
        return base_addresses["USDe"]
    elif asset_type == "sUSDe":
        return base_addresses["sUSDe"]
    elif asset_type == "PT-sUSDe-24Oct2024":
        return base_addresses["PT-sUSDe-24Oct2024"]
    elif asset_type == "PT-sUSDe-26Dec2024":
        return base_addresses["PT-sUSDe-26Dec2024"]
    elif asset_type == "PT-sUSDe-27Mar2025":
        return base_addresses["PT-sUSDe-27Mar2025"]
    else:
        return base_addresses["Idle"]

# Initialize the JSON structure
output_json = []

for allocation_type, row in manual_reallocation_pools.iterrows():
    if row['Net Change'] != 0:
        pool_type = determine_pool_type(allocation_type)
        lltv = str(int(float(allocation_type.split()[1].replace('%', '')) * 10**16))  # Convert percentage to LLTV
        pool = pool_type + [lltv]

        allocation_wei = str(row['New Allocation Wei'])  # Convert to string
        
        output_json.append([pool, allocation_wei])

# Add the 'Idle' pool entry
idle_pool = base_addresses["Idle"] + ["0"]
idle_allocation_wei = "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff"
output_json.append([idle_pool, idle_allocation_wei])

# Convert the list to JSON formatted string
output_str = json.dumps(output_json, indent=2)

# Print or save the JSON formatted string
print(output_str)

# Optionally, save to a file
with open("morpho_vault_realloc.json", "w") as file:
    file.write(output_str)

[
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x9D39A5DE30e57443BfF2A8307A4256c8797A3497",
      "0x5D916980D5Ae1737a8330Bf24dF812b2911Aae25",
      "0x870aC11D48B15DB9a138Cf899d20F13F79Ba00BC",
      "860000000000000000"
    ],
    "69315255000000000000000000"
  ],
  [
    [
      "0x6B175474E89094C44Da98b954EedeAC495271d0F",
      "0x0000000000000000000000000000000000000000",
      "0x0000000000000000000000000000000000000000",
      "0x0000000000000000000000000000000000000000",
      "0"
    ],
    "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff"
  ]
]
